In [1]:
!mkdir Face_regocnition_project && cd Face_regocnition_project

In [2]:
%%writefile Face_regocnition_project/data_preprocessing.py
import os
import zipfile
import shutil
import re

def extract_and_arrange(zip_file="Faces.zip", extracted_folder="Faces_unorganized", structured_folder="Faces"):
    print("Extracting zip file...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("Extraction completed. Files extracted into:", extracted_folder)

    print("Arranging files into structured folders...")
    os.makedirs(structured_folder, exist_ok=True)

    # Recursively walk through the extracted folder
    for root, dirs, files in os.walk(extracted_folder):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Extract label: everything before the underscore followed by digits.
                match = re.match(r'^(.*?)_\d+', filename)
                if match:
                    label = match.group(1)
                else:
                    label = os.path.splitext(filename)[0]

                # Create a subfolder for this label if it doesn't exist.
                label_dir = os.path.join(structured_folder, label)
                os.makedirs(label_dir, exist_ok=True)

                # Define source and destination paths.
                src = os.path.join(root, filename)
                dst = os.path.join(label_dir, filename)
                shutil.move(src, dst)
    print("Dataset arranged in folder:", structured_folder)


Writing Face_regocnition_project/data_preprocessing.py


In [3]:
%%writefile Face_regocnition_project/data_loading.py
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

def get_data_loaders(data_dir="Faces", batch_size=32, image_size=(128,128)):
    # Define transformations: resize, convert to tensor, and normalize.
    data_transforms = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    # Load the dataset from the arranged folder
    dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
    print("Classes found:", dataset.classes)

    # Split the dataset into training and validation sets (80-20 split)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Return datasets, loaders and class names for later use
    return train_dataset, val_dataset, train_loader, val_loader, dataset.classes


Writing Face_regocnition_project/data_loading.py


In [4]:
%%writefile Face_regocnition_project/model_training.py
import time
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, train_dataset, val_dataset, device, criterion, optimizer, scheduler=None):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train(self, num_epochs=10, early_stop_patience=5):
        best_val_loss = float('inf')
        epochs_no_improve = 0

        for epoch in range(num_epochs):
            epoch_start_time = time.time()

            # ----- Training Phase -----
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            train_bar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
            for images, labels in train_bar:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=2.0)
                self.optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                train_bar.set_postfix(loss=loss.item())

            epoch_loss = running_loss / len(self.train_dataset)
            epoch_accuracy = 100 * correct / total
            print(f"\nEpoch [{epoch+1}/{num_epochs}] Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

            # ----- Validation Phase -----
            self.model.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0

            val_bar = tqdm(self.val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation")
            with torch.no_grad():
                for images, labels in val_bar:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()
                    val_bar.set_postfix(loss=loss.item())

            val_loss = val_loss / len(self.val_dataset)
            val_accuracy = 100 * correct_val / total_val
            print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

            if self.scheduler is not None:
                self.scheduler.step(val_loss)
            current_lr = self.optimizer.param_groups[0]['lr']
            print(f"Current Learning Rate: {current_lr}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self.model.state_dict(), 'best_model.pth')
                print("Model checkpoint saved!")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= early_stop_patience:
                print("Early stopping triggered.")
                break

            epoch_duration = time.time() - epoch_start_time
            print(f"Epoch duration: {epoch_duration:.2f} seconds\n")


Writing Face_regocnition_project/model_training.py


In [5]:
%%writefile Face_regocnition_project/inference.py
import torch
from torchvision import transforms
from PIL import Image
import cv2

class InferenceEngine:
    def __init__(self, model, device, transform=None, class_names=None):
        self.model = model
        self.device = device
        self.model.eval()
        self.transform = transform
        self.class_names = class_names if class_names is not None else []

    def predict_image(self, image_path):
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        image = image.unsqueeze(0).to(self.device)
        with torch.no_grad():
            outputs = self.model(image)
            _, predicted = torch.max(outputs.data, 1)
        if self.class_names:
            return self.class_names[predicted.item()]
        return predicted.item()

    def webcam_inference(self, capture_interval=10):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Unable to open the webcam.")
            return
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to capture frame.")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(frame_rgb)
            if self.transform:
                image = self.transform(image)
            image = image.unsqueeze(0).to(self.device)
            with torch.no_grad():
                outputs = self.model(image)
                _, predicted = torch.max(outputs.data, 1)
            label = self.class_names[predicted.item()] if self.class_names else str(predicted.item())
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow("Webcam Inference", frame)
            if cv2.waitKey(capture_interval * 1000) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


Writing Face_regocnition_project/inference.py


In [6]:
%%writefile Face_regocnition_project/main.py
# Add the project folder to the Python path if needed
import sys
sys.path.append("Face_regocnition_project")

# Import your modules
from data_preprocessing import extract_and_arrange
from data_loading import get_data_loaders
from model_training import ModelTrainer
from inference import InferenceEngine

# Run the preprocessing step (if not done already)
extract_and_arrange(zip_file="Faces.zip", extracted_folder="Faces_unorganized", structured_folder="Faces")

# Load your data
train_dataset, val_dataset, train_loader, val_loader, class_names = get_data_loaders(data_dir="Faces", batch_size=32, image_size=(128,128))

# Assume you have defined your model, device, criterion, optimizer, etc.
# For example:
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your model here or load a pre-trained model
# For demonstration, here we create a simple model:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(128*128*3, 100),
    nn.ReLU(),
    nn.Linear(100, len(class_names))
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Initialize your trainer and start training
trainer = ModelTrainer(model, train_loader, val_loader, train_dataset, val_dataset, device, criterion, optimizer, scheduler)
trainer.train(num_epochs=10, early_stop_patience=5)

# (Optional) Use inference
inference_engine = InferenceEngine(model, device, transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
]), class_names=class_names)
prediction = inference_engine.predict_image("path/to/your/image.jpg")
print("Predicted class:", prediction)


Writing Face_regocnition_project/main.py


In [7]:
!zip -r Face_regocnition_project.zip Face_regocnition_project
from google.colab import files
files.download("Face_regocnition_project.zip")

  adding: Face_regocnition_project/ (stored 0%)
  adding: Face_regocnition_project/model_training.py (deflated 73%)
  adding: Face_regocnition_project/data_loading.py (deflated 60%)
  adding: Face_regocnition_project/data_preprocessing.py (deflated 61%)
  adding: Face_regocnition_project/Faces.zip (stored 0%)
  adding: Face_regocnition_project/inference.py (deflated 64%)
  adding: Face_regocnition_project/main.py (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>